# 教学楼&教室节点

In [5]:
from py2neo import Graph, Node, Relationship
import json

class KnowledgeGraphBuilder:
    def __init__(self, uri, user, password):
        self.graph = Graph(uri, auth=(user, password))

    def build_graph(self, rooms_file):
        with open(rooms_file, 'r', encoding='utf-8') as f:
            rooms_data = json.load(f)

        tx = self.graph.begin()
        campuses = {}  # 缓存校区节点 {name: node}
        buildings = {}  # 缓存教学楼节点 {name: node}
        classrooms = set()  # 避免重复教室

        try:
            for room_data in rooms_data:
                campus_name = room_data["校区"]
                building_name = room_data["教学楼"]
                classroom_name = room_data["教室名称"]
                seats = int(room_data["上课座位数"])
                exam_seats = int(room_data.get("考试座位数", 0))  # 处理可能缺失的字段

                # 获取校区节点（假设校区已存在）
                campus_node = self.graph.nodes.match("Campus", name=campus_name).first()
                if not campus_node:
                    raise ValueError(f"校区 {campus_name} 不存在，请先创建校区节点")

                # ================= 处理教学楼节点 =================
                # 先检查内存缓存
                building_node = buildings.get(building_name)
                
                if not building_node:
                    # 检查数据库是否已存在
                    building_node = self.graph.nodes.match("Building", name=building_name).first()
                    if building_node:
                        print(f"教学楼 {building_name} 已存在，跳过创建")
                        buildings[building_name] = building_node
                    else:
                        # 创建新教学楼节点
                        print(f"创建教学楼节点 {building_name}")
                        building_node = Node("Building", name=building_name)
                        tx.create(building_node)
                        # 创建校区关系
                        tx.create(Relationship(building_node, "LOCATED_IN", campus_node))
                        buildings[building_name] = building_node

                # ================= 处理教室节点 =================
                if classroom_name not in classrooms:
                    # 检查数据库是否已存在
                    classroom_node = self.graph.nodes.match("Classroom", name=classroom_name).first()
                    
                    if not classroom_node:
                        # 创建新教室节点
                        classroom_node = Node(
                            "Classroom",
                            name=classroom_name,
                            seats=seats,
                            exam_seats=exam_seats
                        )
                        tx.create(classroom_node)
                        # 创建教学楼关系
                        tx.create(Relationship(classroom_node, "BELONGS_TO", building_node))
                    
                    # 加入已处理集合（无论是否新建）
                    classrooms.add(classroom_name)

            tx.commit()
        except Exception as e:
            tx.rollback()
            print(f"发生错误: {e}")

if __name__ == "__main__":
    kg_builder = KnowledgeGraphBuilder(
        uri="bolt://localhost:7687", 
        user="neo4j",
        password="neo4j123456"
    )
    kg_builder.build_graph("./rooms.json")

创建教学楼节点 建筑学院
创建教学楼节点 文渊楼
创建教学楼节点 北学楼
创建教学楼节点 地科楼
创建教学楼节点 培训楼
创建教学楼节点 怀周楼
创建教学楼节点 体育馆
创建教学楼节点 校工会形体室
创建教学楼节点 室外
创建教学楼节点 东二院
创建教学楼节点 东一院
创建教学楼节点 国重楼
创建教学楼节点 材料学院
创建教学楼节点 格物楼2栋
创建教学楼节点 格物楼1栋
创建教学楼节点 力行楼
创建教学楼节点 中山邦翰楼
创建教学楼节点 文汇楼3栋
创建教学楼节点 文汇楼2栋
创建教学楼节点 楠苑体育场
教学楼 信息学院 已存在，跳过创建
教学楼 地球科学学院 已存在，跳过创建
创建教学楼节点 物理学院
创建教学楼节点 化工学院
教学楼 软件学院 已存在，跳过创建
创建教学楼节点 生科学院
创建教学楼节点 数统学院
教学楼 经济学院 已存在，跳过创建
教学楼 新闻学院 已存在，跳过创建
创建教学楼节点 商旅学院
教学楼 医学院 已存在，跳过创建
创建教学楼节点 格物楼4栋
创建教学楼节点 东二院2栋
创建教学楼节点 东二院3栋
创建教学楼节点 北院后勤集团
创建教学楼节点 东门文体中心
创建教学楼节点 附属医院
创建教学楼节点 文津楼
创建教学楼节点 泽清堂
创建教学楼节点 图书馆（东陆）
创建教学楼节点 本部工会楼
创建教学楼节点 映秋院
创建教学楼节点 公管学院
创建教学楼节点 格物楼3栋
创建教学楼节点 庆来堂
创建教学楼节点 工程实训楼
创建教学楼节点 心理健康咨询服务中心（体育学院楼）
创建教学楼节点 工程中心东C栋
创建教学楼节点 综合实验实训楼


C:\Users\Burning\AppData\Local\Temp\ipykernel_21416\1531976567.py:69: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  tx.commit()


In [2]:
import json
import re
from py2neo import Graph, Node, Relationship, NodeMatcher

# 连接 Neo4j 数据库，修改连接地址和认证信息
graph = Graph("bolt://localhost:7687", auth=("neo4j", "neo4j123456"))
matcher = NodeMatcher(graph)

# 节次到具体时间的映射表
PERIOD_MAP = {
    "1-2节": ("08:30", "10:10"),
    "3-4节": ("10:30", "12:10"),
    "5-6节": ("14:00", "15:40"),
    "7-8节": ("16:00", "17:40"),
    "9-10节": ("19:00", "20:40")
}

def parse_time_place(time_place_str):
    """
    解析时间地点字段，拆分成多个上课时段，并根据 PERIOD_MAP 获取对应的开始和结束时间。
    返回的 sessions 列表中，每个字典包含：
      - week_range: 如 "1-17"
      - weekday: 如 "星期一"
      - time_period: 如 "7-8节"
      - raw: 原始时间地点描述
      - classroom: 教室名称，如 "文汇楼2栋2413"
      - start_time, end_time: 若能从 PERIOD_MAP 匹配上，则对应的具体时间
    """
    sessions = []
    # 以英文逗号和中文逗号分割多个时段
    time_place_list = re.split(r'[,，]', time_place_str)
    for tp in time_place_list:
        tp = tp.strip()
        # 提取周次范围，例如 "1-17周"
        week_match = re.search(r'(\d+-\d+)周', tp)
        week_range = week_match.group(1) if week_match else None

        # 提取星期，例如 "星期一"
        weekday_match = re.search(r'(星期[一二三四五六日])', tp)
        weekday = weekday_match.group(1) if weekday_match else None

        # 提取节次，例如 "7-8节"
        time_match = re.search(r'(\d+-\d+节)', tp)
        time_period = time_match.group(1) if time_match else None

        # 根据时间映射表获取上课开始和结束时间
        start_time, end_time = None, None
        if time_period in PERIOD_MAP:
            start_time, end_time = PERIOD_MAP[time_period]

        # 提取教室名称：假设教室名称在“节”之后的部分
        classroom_match = re.search(r'节\s*(.+)', tp)
        classroom = classroom_match.group(1).strip() if classroom_match else None

        sessions.append({
            'week_range': week_range,
            'weekday': weekday,
            'time_period': time_period,
            'raw': tp,
            'classroom': classroom,
            'start_time': start_time,
            'end_time': end_time
        })
    return sessions
def import_courses_new(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        courses = json.load(f)

    for record in courses:
        course_id = record["课程号"]
        course_name = record["课程名"]
        section = record["课序号"]

        # 1️⃣ 创建或复用 Course（抽象课程）节点
        course_node = matcher.match("Course", courseId=course_id).first()
        if not course_node:
            course_node = Node("Course", courseId=course_id, courseName=course_name)
            graph.create(course_node)

        # 2️⃣ 创建或复用 CourseSession（教学班）节点
        session_node = matcher.match("CourseSession", courseId=course_id, section=section).first()
        if not session_node:
            session_node = Node("CourseSession",
                                name="教学班"+str(section),
                                courseName=course_name,
                                courseId=course_id,
                                department=record["开课单位"],
                                hours=int(record["学时"]) if record["学时"] else 0,
                                credits=float(record["学分"]) if record["学分"] else 0,
                                enrollment=int(record["选课人数"]) if record["选课人数"] else 0,
                                campus=record["校区"],
                                courseCategory=record.get("选课类别", ""))
            graph.create(session_node)
            # 课程 -> 教学班
            graph.create(Relationship(session_node, "OFFERED_AS", course_node))
        else:
            print(f"复用已有教学班：{course_id}-{section}")

        # 3️⃣ 处理上课时间 → 多个 Meeting
        for i,s in enumerate(parse_time_place(record["时间地点"])):
            meeting_node = Node("Meeting",
                                name="上课时段"+str(i+1),
                                weekRange=s["week_range"],
                                weekday=s["weekday"],
                                timePeriod=s["time_period"],
                                rawTimePlace=s["raw"],
                                start_time=s["start_time"],
                                end_time=s["end_time"])
            graph.create(meeting_node)
            graph.create(Relationship(session_node, "HAS_MEETING", meeting_node))

            # 处理教室
            if s["classroom"]:
                classroom_node = matcher.match("Classroom", name=s["classroom"]).first()
                if not classroom_node:
                    classroom_node = Node("Classroom", name=s["classroom"])
                    graph.create(classroom_node)
                graph.create(Relationship(meeting_node, "AT_CLASSROOM", classroom_node))

        # 4️⃣ 教师
        teacher_names = record["上课教师"].split(',') if ',' in record["上课教师"] else [record["上课教师"]]
        for teacher_name in teacher_names:
            teacher_node = matcher.match("Person", "Faculty", name=teacher_name).first()
            if not teacher_node:
                teacher_node = Node("Person", "Faculty", name=teacher_name)
                graph.create(teacher_node)
            graph.create(Relationship(session_node, "TAUGHT_BY", teacher_node))


def import_courses(file_path):
    # 读取 courses.json 文件（确保文件编码为 utf-8）
    with open(file_path, "r", encoding="utf-8") as f:
        courses = json.load(f)

    for record in courses:
        # 创建 CourseSession 节点（记录课程基础信息）
        course_node = Node("CourseSession",
                           courseId=record["课程号"],
                           courseName=record["课程名"],
                           section=record["课序号"],
                           department=record["开课单位"],
                           # 下面的字段如果为空则赋值为0
                           
                           hours=int(record["学时"]) if record["学时"] else 0,
                           credits=float(record["学分"]) if record["学分"] else 0,
                           enrollment=int(record["选课人数"]) if record["选课人数"] else 0,
                           campus=record["校区"],
                           courseCategory=record.get("选课类别", ""))
        graph.create(course_node)

        # 解析“时间地点”字段，拆分出多个上课时段
        sessions = parse_time_place(record["时间地点"])
        for s in sessions:
            # 创建 Meeting 节点，同时记录解析后的时间信息
            meeting_node = Node("Meeting",
                                weekRange=s["week_range"],
                                weekday=s["weekday"],
                                timePeriod=s["time_period"],
                                rawTimePlace=s["raw"],
                                start_time=s["start_time"],
                                end_time=s["end_time"])
            graph.create(meeting_node)
            # 建立 CourseSession 与 Meeting 的 HAS_MEETING 关系
            graph.create(Relationship(course_node, "HAS_MEETING", meeting_node))

            # 根据教室名称查找已有的 Classroom 节点，并建立 AT_CLASSROOM 关系
            if s["classroom"]:
                classroom_node = matcher.match("Classroom", name=s["classroom"]).first()
                if classroom_node:
                    graph.create(Relationship(meeting_node, "AT_CLASSROOM", classroom_node))
                else:
                    print(f"创建新教室节点：{s['classroom']}")
                    classroom_node = Node("Classroom", name=s["classroom"])
                    graph.create(classroom_node)
                    graph.create(Relationship(meeting_node, "AT_CLASSROOM", classroom_node))

        # 根据“上课教师”属性，查找已有的 Teacher 节点，并建立 TAUGHT_BY 关系
        teacher_name = record["上课教师"]
        if ',' in teacher_name:
            teacher_names = teacher_name.split(',')
        else:
            teacher_names = [teacher_name]
        for teacher_name in teacher_names:
            teacher_node = matcher.match("Person", "Faculty", name=teacher_name).first()
            if teacher_node:
                graph.create(Relationship(course_node, "TAUGHT_BY", teacher_node))
                print(f"教师 {teacher_name} 已存在，无需创建")
            else:
                print(f"创建新教师节点：{teacher_name}")
                teacher_node = Node("Person", "Faculty", name=teacher_name)
                graph.create(teacher_node)
                graph.create(Relationship(course_node, "TAUGHT_BY", teacher_node))

if __name__ == "__main__":
    # 调用导入函数，确保 courses.json 文件路径正确
    import_courses_new("./courses.json")
    """
    概念	节点	唯一依据	关联
    抽象课程	Course	课程号	-[:OFFERED_AS]->(CourseSession)
    教学班	CourseSession	课程号 + 课序号	-[:HAS_MEETING]->(Meeting)
    上课时段	Meeting	解析后的时间地点	-[:AT_CLASSROOM]->(Classroom)
    教师	Person:Faculty	教师姓名	-[:TAUGHT_BY]->(CourseSession)
    教室	Classroom	教室名称	与 Meeting 关联
    """